In [1]:
'''
Generation ETH in dataframe format

'/cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/CANCER_eth/commit_c4dd02c_conf2_Frame_cap0_runs/TCGA_Ovarian_374/filtering_intermediate/complete_cancer_candidates_order_r_complete.tsv.gz'


Generation OHSU in dataframe format
Breast/Ovarian
/cluster/work/grlab/projects/projects2020_OHSU/share_OHUS_PNLL/OHSU_June2023_filter-debug_complete-annotated-shortlist.tsv.gz
'''


"\nGeneration ETH in dataframe format\n\n'/cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/CANCER_eth/commit_c4dd02c_conf2_Frame_cap0_runs/TCGA_Ovarian_374/filtering_intermediate/complete_cancer_candidates_order_r_complete.tsv.gz'\n\n\nGeneration OHSU in dataframe format\nBreast/Ovarian\n/cluster/work/grlab/projects/projects2020_OHSU/share_OHUS_PNLL/OHSU_June2023_filter-debug_complete-annotated-shortlist.tsv.gz\n"

In [1]:
# %load /cluster/home/myurchikova/github/projects2020_ohsu/eth/learning_Master_thesis/TASKS/func/base_imports.py
import pandas as pd
import numpy as np
import tqdm 
import glob
import os
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import tarfile
import re
from matplotlib_venn import venn2, venn2_circles, venn2_unweighted
from matplotlib_venn import venn3, venn3_circles
import sys
from matplotlib.ticker import ScalarFormatter
sys.path.append(r"/cluster/home/myurchikova/github/projects2020_ohsu/eth/learning_Master_thesis/TASKS/func")
from config import *
from functions import *
import time
import pathlib

In [2]:
#ETH Ovarian
PATH_ETH_Ovarian = '/cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/CANCER_eth/commit_c4dd02c_conf2_Frame_cap0_runs/TCGA_Ovarian_374/cohort_mutNone'

In [3]:
#OHSU Ovarian
PATH_OHSU_Ovarian = '/cluster/work/grlab/projects/projects2020_OHSU/share_OHUS_PNLL/OHSU_June2023_filter-debug_complete-annotated-shortlist.tsv.gz'#'/cluster/work/grlab/projects/projects2020_OHSU/share_OHUS_PNLL/June28_renamed_kmerfiles_OHSU.tar.gz'#'/cluster/work/grlab/projects/projects2020_OHSU/share_OHUS_PNLL/OHSU_June2023_filter-debug_complete-annotated-shortlist.tsv.gz'

In [4]:
ohsu_df=pd.read_csv(PATH_OHSU_Ovarian, sep = '\t', low_memory=False) #. read_csv() function takes a path to a CSV file and reads the data into a Pandas DataFrame object Despite the name, a CSV doesn't need to rely on commas as the separator between pieces of information. This separator, called a delimiter, can be a semicolon, space, or some other character, though the comma is most common.
ohsu_df.rename(columns = {'in-frame_neoepitopes': 'kmer'}, inplace = True)
# ohsu_df#[['gene_id', 'kmer']]

In [ ]:
# set start time
s2 =time.time()

## Configuration 
# LOG => True:False
# EXPRESSIONS => The samples of interest
# COUNT_OF_ENTRY => ineger (-1 if all records)
LOG=False
COUNT_OF_ENTRY = -1

EXPRESSIONS = ['TCGA25131901A01R156513',
           'TCGA25131301A01R156513',
           'TCGA61200801A02R156813',
           'TCGA24143101A01R156613',
           'TCGA24229801A01R156913']

# Variable declaration
tt='\n--------------------------------------------------------\n'
PATTERN = "*.gz"
RE_PATTERN="/tmp_out_ref_batch_[\d+]{1,7}\/"
gen_eth =  []
inter = []
eth_without_ohsu =  []
ohsu_withou_eth =  []
fs_oshu =  []
fs_eth =  []
batch =  []
gene_id =  []
path_to_tab_eth = []
df_kmer_common={}

MAIN_COLS = ['gene_id', 'kmer'] + EXPRESSIONS
out_table = {
    'gene_id': gene_id,
    'batch':batch,
    'eth_without_ohsu':eth_without_ohsu,
    'ohsu_withou_eth':ohsu_withou_eth,
    'inter':inter,
    'full_size_oshu': fs_oshu,
    'full_size_eth': fs_eth,
}

batch_to_gene = pd.read_csv('/cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/CANCER_eth/commit_c4dd02c_conf2_Frame_cap0_runs/TCGA_Breast_1102/batch_to_gene.txt', 
                           names = ['gene', 'batch'])
ETH_FOLDER = PATH_ETH_Ovarian
OHSU_PATH = PATH_OHSU_Ovarian
ohsu_df=pd.read_csv(OHSU_PATH, sep = '\t', low_memory=False) #. read_csv() function takes a path to a CSV file and reads the data into a Pandas DataFrame object Despite the name, a CSV doesn't need to rely on commas as the separator between pieces of information. This separator, called a delimiter, can be a semicolon, space, or some other character, though the comma is most common.
ohsu_df.rename(columns = {'in-frame_neoepitopes': 'kmer'}, inplace = True)

ohsu_df = table_processing.change_column_names(ohsu_df)
ohsu_df = table_processing.preprocess_ohsu(ohsu_df)



path_eth = glob.glob(ETH_FOLDER + '/*/*Junc*')

# Set before for loop time
se=time.time()

# This loop finds each table in the path (GEN)
for path in path_eth if COUNT_OF_ENTRY == -1 else path_eth[:COUNT_OF_ENTRY]:
    path = pathlib.Path(path)

    # Find number of gene
    batch_ETH=re.findall('[\d+]{1,7}',(re.findall(RE_PATTERN,str(path)))[0])[0]
    
    if LOG == True:
        print(path)
        print(batch_ETH)
    # Find name of gene
    try:
        bh=batch_to_gene.loc[batch_to_gene['batch'] == int(batch_ETH), 'gene']
        gene_ETH=bh.values[0]
    except:
        print("BH",tt,bh)
     
    # Reading Gene (with ETH) from OSHU table & create OSHU kmers set

# print(ohsu_kmers)
    
    # In general we have one table
    next=False
    is_table=False
    # If we have more than one table from directory we take into loop all avaliable gen tables
    for tab in path.glob(PATTERN): 
        #print(tab)
        is_table=True
        # Reading & Renaming table to our standarts
        ETH_df=pd.read_csv(str(tab), sep = '\t', low_memory=False) #, names=['in-frame_neoepitopes','TCGAC8A12P01A11RA11507all','TCGAAOA0JM01A21RA05607all','TCGABHA18V01A11RA12D07all','TCGAA2A0D201A21RA03407all','TCGAA2A0SX01A12RA08407all']

        ETH_df.rename(columns = {'in-frame_neoepitopes': 'kmer',
                                 'TCGA25131901A01R156513all':'TCGA25131901A01R156513',
                                 'TCGA25131301A01R156513all':'TCGA25131301A01R156513',
                                 'TCGA61200801A02R156813all':'TCGA61200801A02R156813',
                                 'TCGA24143101A01R156613all':'TCGA24143101A01R156613',
                                 'TCGA24229801A01R156913all':'TCGA24229801A01R156913'}, inplace = True)
        #print(ETH_df)
        
        
        # For the samples of interest
        for exp_col in tqdm.tqdm(EXPRESSIONS):
            try:
                # If we have more one or one table
                if next==True:
                    eth_kmers = pd.concat([eth_kmers,ETH_df[ETH_df[exp_col] > 0]['kmer']])
                    ohsu_df = pd.concat([ohsu_df,ohsu_df[ohsu_df[exp_col] > 0]['kmer']])
                else:
                    eth_kmers = ETH_df[ETH_df[exp_col] > 0]['kmer']
                    ohsu_df = ohsu_df[ohsu_df[exp_col] > 0]
                    next=True
                #print(eth_kmers)
            except:
                print("Error Exppression")
            
    ohsu_df=ohsu_df.iloc[:,ohsu_df.columns.get_indexer(MAIN_COLS)]
    ohsu_df_1gene = ohsu_df.loc[ohsu_df['gene_id'] == gene_ETH]
    ohsu_kmers = set(ohsu_df_1gene['kmer'])
            
            
    if is_table:
        # Append information about one Gene to finall structure
        batch.append(batch_ETH)
        gene_id.append(gene_ETH)
        
        eth_kmers=set(eth_kmers)
        
        fs_oshu.append(n_ohsu:= len(ohsu_kmers))
        fs_eth.append(n_eth:=len(eth_kmers))
        
        eth_without_ohsu.append(len(eth_kmers.difference(ohsu_kmers)))
        ohsu_withou_eth.append(len(ohsu_kmers.difference(eth_kmers)))
        
        inter.append(len(eth_kmers & ohsu_kmers))

        #print('GEN Record:\t',len(gene_id))

        # Log information
        if LOG == True:
            print(tt,'GEN:\n',gene_ETH,tt)
            print(tt,'OHSU COUNT:\n',n_ohsu,tt)
            print(tt,'OHSU KMERS:\n',ohsu_kmers,tt)
            print(tt,'OHSU KMERS:\n',len(ohsu_kmers),tt)
            print(tt,'ETH COUNT:\n',n_eth,tt)
            print(tt,'ETH KMERS:\n',eth_kmers,tt)
    
    
# Convert structure to Data Frame
out_df=pd.DataFrame(data=out_table)

# Set end time
e2=time.time()



# Log information
if LOG == True:
    print('Loop time\t',e2-se)
print('Code time:\t',e2-s2)

## TCGA25131901A01R156513

100%|█████████████████████████████████████████████| 5/5 [00:01<00:00,  2.91it/s]


BH 
--------------------------------------------------------
 Series([], Name: gene, dtype: object)


100%|█████████████████████████████████████████████| 5/5 [00:00<00:00,  9.94it/s]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00,  6.44it/s]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00,  6.43it/s]


...

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00,  5.86it/s]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00,  6.13it/s]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:01<00:00,  4.50it/s]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00,  6.32it/s]


Code time:	 115616.51721191406


In [ ]:
path_ovarian=create_path.create_path(SAVE_DIR,[DIR_CSV,DIR_OVARIAN,NAME_TABLES,NAME_NON_FILTERING_OVARIAN])
out_df.to_csv(path_ovarian, header = True, sep='\t')